This helper makes sure that the packages that rebuilt successfully with Python 3.10 but fail to install due to their dependencies not yet rebuilt have bugzillas blocked by the appropriate bugzillas of said dependencies.

 1. For every package that has been successfully rebuilt, we see if it has a bugzilla that blocks `PYTHON3.10` and `F35FailsToInstall`.
 2. If so, we parse the initial comment for missing `python3.10dist()` dependencies.
 3. We convert the dependencies to `python3.9dist()` dependencies and find their component.
 4. If the component has a `PYTHON3.10` bugzilla, we mark it as blocking the bugzilla from (1).

In [1]:
from IPython.display import display, Markdown

In [2]:
import bugzilla
bzapi = bugzilla.Bugzilla('bugzilla.redhat.com')
query = bzapi.build_query(product='Fedora')
query['blocks'] = 2135404  # PYTHON3.12
query['limit'] = 20 # Bugzilla page size
query['offset'] = 0
query['status'] = '__open__'
bugz = []
while len(partial := bzapi.query(query)) == 20:
    bugz += partial
    query['offset'] += 20
    print(len(bugz))
bugz += partial

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820


In [3]:
F39FailsToInstall = 2168845

In [4]:
python312 = !cat python312.pkgs

In [5]:
len(pure_fti_bugz := [b for b in bugz if b.component in python312 and F39FailsToInstall in b.blocks])
pure_fti_bugz

[<Bug #2219921 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4b88d50>,
 <Bug #2219924 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4b93550>,
 <Bug #2219928 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4ba18d0>,
 <Bug #2219931 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4ba7190>,
 <Bug #2219937 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4bbe5d0>,
 <Bug #2219959 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4bf23d0>,
 <Bug #2219962 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4c02290>,
 <Bug #2219999 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4c5e210>,
 <Bug #2220002 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4c6add0>,
 <Bug #2220019 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4a9dd10>,
 <Bug #2220057 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4aff890>,
 <Bug #2220059 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4b08a90>,
 <Bug #2220066 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f8dc4b1f650>,

In [15]:
for bug in pure_fti_bugz:
    display(Markdown(f'# {bug.component}\n[rbz#{bug.id}](https://bugzilla.redhat.com/{bug.id})'))
    comments = bug.getcomments()
    initial_comment = comments[0]
    if initial_comment['creator'] != 'fti-bugs@fedoraproject.org':
        print('Not an automated FTI bug by Miro')
        continue

    blocked_by = set()
    lines = initial_comment['text'].splitlines()
    for line in lines:
        if line.startswith('  - nothing provides '):
            print(line)
            provide = line.split(' ')[5]
            if provide.startswith('('):
                provide = provide[1:]
            if provide.startswith('python3.12dist('):
                old_provide = provide.replace('python3.12dist(', 'python3.11dist(')
                blocker_components = !repoquery -q --repo=rawhide --source --whatprovides "{old_provide}" | pkgname
                if len(set(blocker_components)) == 1:
                    blocker_component = blocker_components[0]
                    blocker_bugz = [b for b in bugz if b.component == blocker_component]
                    if len(blocker_bugz) > 1:
                        print(f"    = too many bugzillas for {blocker_component}")
                        for bug in blocker_bugz:
                            print(f"        https://bugzilla.redhat.com/{bug.id}")
                    elif blocker_bugz:
                        bid = blocker_bugz[0].id
                        print(f"    = blocked by {blocker_component} https://bugzilla.redhat.com/{bid}")
                        blocked_by.add(bid)
                else:
                    print('Check:', blocker_components)
    if blocked_by:
        add_blockers = bzapi.build_update(depends_on_add=sorted(blocked_by))
        bzapi.update_bugs([bug.id], add_blockers)

# Mayavi
[rbz#2219921](https://bugzilla.redhat.com/2219921)

  - nothing provides python3.12dist(envisage) needed by python3-mayavi-4.8.1-4.fc39.x86_64
    = blocked by python-envisage https://bugzilla.redhat.com/2220207
  - nothing provides python3.12dist(pyface) >= 6.1.1 needed by python3-mayavi-4.8.1-4.fc39.x86_64
    = blocked by python-pyface https://bugzilla.redhat.com/2220427
  - nothing provides python3.12dist(traitsui) >= 7 needed by python3-mayavi-4.8.1-4.fc39.x86_64
    = blocked by python-traitsui https://bugzilla.redhat.com/2220539


# OpenLP
[rbz#2219924](https://bugzilla.redhat.com/2219924)

  - nothing provides python3.12dist(flask-cors) needed by OpenLP-3.0.2-2.fc39.noarch
    = blocked by python-flask-cors https://bugzilla.redhat.com/2220233


# aiodnsbrute
[rbz#2219928](https://bugzilla.redhat.com/2219928)

...

  - nothing provides python3.12dist(pyzolib) needed by python3-iep-3.7-26.fc39.noarch
    = too many bugzillas for python-pyzolib
        https://bugzilla.redhat.com/2154987
        https://bugzilla.redhat.com/2220469


# resalloc-openstack
[rbz#2220589](https://bugzilla.redhat.com/2220589)

  - nothing provides python3.12dist(python-neutronclient) needed by resalloc-openstack-9.6-2.fc39.noarch
    = blocked by python-neutronclient https://bugzilla.redhat.com/2220355


# spyder
[rbz#2220598](https://bugzilla.redhat.com/2220598)

  - nothing provides (python3.12dist(python-lsp-server) < 1.5~~ with python3.12dist(python-lsp-server) >= 1.4.1) needed by python3-spyder-5.3.1-7.fc39.noarch
    = blocked by python-lsp-server https://bugzilla.redhat.com/2220316
  - nothing provides (python3.12dist(qdarkstyle) < 3.1~~ with python3.12dist(qdarkstyle) >= 3.0.2) needed by python3-spyder-5.3.1-7.fc39.noarch
    = blocked by python-qdarkstyle https://bugzilla.redhat.com/2220472
  - nothing provides (python3.12dist(spyder-kernels) < 2.4~~ with python3.12dist(spyder-kernels) >= 2.3.1) needed by python3-spyder-5.3.1-7.fc39.noarch
    = blocked by python-spyder-kernels https://bugzilla.redhat.com/2220520
  - nothing provides python3.12dist(diff-match-patch) >= 20181111 needed by python3-spyder-5.3.1-7.fc39.noarch
    = blocked by python-diff-match-patch https://bugzilla.redhat.com/2220190
  - nothing provides python3.12dist(python-lsp-black) >= 1.2 needed by python3-spyder-5.3.1-7.fc39.noarch
    = blocked by python-lsp-black h